<a href="https://colab.research.google.com/github/shiki-fujin/DS-MLE-exam/blob/main/text_classification_sampleDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

Installations

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [5]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


For Hyperparam search

In [6]:
! pip install optuna
! pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 34.6 MB/s eta 0:00:00


Importing Utility libraries

In [7]:
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML

Importing Main libraries

In [8]:
from huggingface_hub import notebook_login
import transformers
import datasets

Verify version of Transformers : at least 4.11.0

In [9]:
print(transformers.__version__)

4.25.1


Obtain then store Hugging Face authentication token

In [10]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


## Trying load_dataset() and load_metric()

In [9]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

task determines what **dataset** and **metric** to use


In [15]:
task = "cola"
actual_task = "mnli" if task == "mnli-mm" else task

dataset = datasets.load_dataset("glue", actual_task) # loads the desired dataset
metric = datasets.load_metric('glue', actual_task) # loads the corresponding metric for a specific task

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-15-c6163eaa34cd>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('glue', actual_task) # loads the corresponding metric for a specific task


Peeking at the dataset

In [11]:
dataset["train"][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

In [12]:
# pick random entries in the dataset
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


show_random_elements(dataset["train"])

,sentence,label,idx
0,The critics laughed the play off the stage.,acceptable,10
1,You must not simply not work.,acceptable,339
2,"As John eats more, keep your mouth shut tighter, OK?",acceptable,133
3,There is a hole in John's quilt's upper right-hand corner.,unacceptable,1888
4,John is the kind of fool that I told you about.,acceptable,5432
5,be eager to leave the meeting.,acceptable,3885
6,"Kim likes Sandy, and Lee Leslie. to try to go to Rome.",acceptable,7087
7,Sandy sang me a song.,acceptable,2959
8,John deposited some money in the checking account and Mary did the same thing in the savings account.,unacceptable,3905
9,"That guy, who I think might be drunk, just hit me!",acceptable,6180


In [13]:
show_random_elements(dataset["validation"], 3)

,sentence,label,idx
0,"Tom picked these grapes, and I washed some turnips, and Suzie will prepare these grapes.",acceptable,84
1,"I can't believe Fred won't, either.",acceptable,49
2,John went home.,acceptable,43


Trying the compute method

Result shows a **matthews_correlation** since the **metric** was instantiated with **task = 'cola'**

In [14]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'matthews_correlation': 0.14060420405767388}

## Trying the Preprocessing method

model_checkpoint will indicate :

*  Tokenizer
*  pre-trained model



In [11]:
model_checkpoint = "distilbert-base-uncased" # specifies which tokenizer to use

In [12]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer('Death can have me when it earns me')

{'input_ids': [101, 2331, 2064, 2031, 2033, 2043, 2009, 7796, 2015, 2033, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [18]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][3][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][3][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][3][sentence2_key]}")

Sentence: The more we study verbs, the crazier they get.


In [19]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)


preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [20]:
# Apply preprocessing function to the whole dataset
# This simply adds the corresponding 'input_ids' and 'attention_mask' to each row
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [23]:
encoded_dataset['train'][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0,
 'input_ids': [101,
  2256,
  2814,
  2180,
  1005,
  1056,
  4965,
  2023,
  4106,
  1010,
  2292,
  2894,
  1996,
  2279,
  2028,
  2057,
  16599,
  1012,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Fine Tuning

The Warning indicates that we don't have pretrained weights --> so we have to Fine-Tune first before inferencing

In [24]:
# Instantiating the model
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

TrainingArguments - configures the training parameters

In [21]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
hub_model_name = f"{model_name}-finetuned-{task}"

args = transformers.TrainingArguments(
    hub_model_name,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Create a function that will compute the predictions

In [22]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Pass the configured training **parameters** & created **function**  

In [27]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = transformers.Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/255M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

Download file runs/Jan21_11-13-45_1304499cffa7/events.out.tfevents.1674299697.1304499cffa7.102.0: 100%|#######…

Download file runs/Jan21_08-10-10_3d25bd44b2fe/1674288651.2198653/events.out.tfevents.1674288651.3d25bd44b2fe.…

...

Clean file runs/Jan21_11-13-45_1304499cffa7/events.out.tfevents.1674299697.1304499cffa7.102.0:  20%|#9        …

Clean file runs/Jan21_08-10-10_3d25bd44b2fe/1674288651.2198653/events.out.tfevents.1674288651.3d25bd44b2fe.605…

Clean file runs/Jan21_11-13-45_1304499cffa7/1674299697.9857948/events.out.tfevents.1674299697.1304499cffa7.102…

Clean file runs/Jan21_08-10-10_3d25bd44b2fe/events.out.tfevents.1674288651.3d25bd44b2fe.605.0:  18%|#8        …

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8551
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2675
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` 

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.525900,0.528932,0.427992
2,0.347300,0.512274,0.478350
3,0.237000,0.551713,0.523522
4,0.173600,0.783125,0.537362
5,0.125500,0.810857,0.536115


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-cola/checkpoint-535
Configuration saved in distilbert-base-uncased-finetuned-cola/checkpoint-535/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola/checkpoint-535/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/checkpoint-535/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/checkpoint-535/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/sp

TrainOutput(global_step=2675, training_loss=0.2707629836608316, metrics={'train_runtime': 5933.1449, 'train_samples_per_second': 7.206, 'train_steps_per_second': 0.451, 'total_flos': 229309863736728.0, 'train_loss': 0.2707629836608316, 'epoch': 5.0})

In [32]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16


{'eval_loss': 0.7831246256828308,
 'eval_matthews_correlation': 0.5373623427702773,
 'eval_runtime': 34.159,
 'eval_samples_per_second': 30.534,
 'eval_steps_per_second': 1.932,
 'epoch': 5.0}

Upload result of training to Hub

In [33]:
trainer.push_to_hub()

Saving model checkpoint to distilbert-base-uncased-finetuned-cola
Configuration saved in distilbert-base-uncased-finetuned-cola/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/special_tokens_map.json


Upload file runs/Jan21_12-59-20_e1996c995beb/events.out.tfevents.1674306523.e1996c995beb.219.0: 100%|#########…

Upload file runs/Jan21_12-59-20_e1996c995beb/events.out.tfevents.1674313582.e1996c995beb.219.2: 100%|#########…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola
   0963b59..f7c7b08  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola
   0963b59..f7c7b08  main -> main

To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola
   f7c7b08..47899cd  main -> main

   f7c7b08..47899cd  main -> main



'https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola/commit/f7c7b083d618f78c3ff2ca5fc44b28cb7e37511d'

Try to instantiate the model

In [27]:
model_from_hub = transformers.AutoModelForSequenceClassification.from_pretrained(f'naksu/{hub_model_name}')

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Hyperparam search

In [32]:
def model_init():
    return transformers.AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
  

train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10)
trainer = transformers.Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBert

Download file pytorch_model.bin:   0%|          | 8.00k/255M [00:00<?, ?B/s]

Download file runs/Jan21_12-59-20_e1996c995beb/events.out.tfevents.1674306523.e1996c995beb.219.0: 100%|#######…

...

Clean file runs/Jan21_12-59-20_e1996c995beb/events.out.tfevents.1674330423.e1996c995beb.219.11:  23%|##2      …

Clean file training_args.bin:  30%|##9       | 1.00k/3.37k [00:00<?, ?B/s]

Clean file runs/Jan21_12-59-20_e1996c995beb/events.out.tfevents.1674345912.e1996c995beb.219.19:  25%|##5      …

Clean file runs/Jan21_12-59-20_e1996c995beb/events.out.tfevents.1674313582.e1996c995beb.219.2: 100%|##########…

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [33]:
best_run = trainer.hyperparameter_search(n_trials=5, direction="maximize")

[I 2023-01-22 03:34:26,797] A new study created in memory with name: no-name-880f845a-c8d0-47a1-9f29-c57105c04778
Trial: {'learning_rate': 1.3672930251700722e-05, 'num_train_epochs': 2, 'seed': 25, 'per_device_train_batch_size': 4}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.b

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.587485,0.000000
2,No log,0.577256,0.202565


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-cola/run-0/checkpoint-214
Configuration saved in distilbert-base-uncased-finetuned-cola/run-0/checkpoint-214/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola/run-0/checkpoint-214/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/run-0/checkpoint-214/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/run-0/checkpoint-214/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/tokenizer_config.json
Special tokens file saved in distilbert-

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.615598,0.000000
2,No log,0.609778,0.000000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-cola/run-1/checkpoint-214
Configuration saved in distilbert-base-uncased-finetuned-cola/run-1/checkpoint-214/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola/run-1/checkpoint-214/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/run-1/checkpoint-214/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/run-1/checkpoint-214/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/tokenizer_config.json
Special tokens file saved in distilbert-

...

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-cola/run-2/checkpoint-214
Configuration saved in distilbert-base-uncased-finetuned-cola/run-2/checkpoint-214/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola/run-2/checkpoint-214/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/run-2/checkpoint-214/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/run-2/checkpoint-214/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/tokenizer_config.json
Special tokens file saved in distilbert-

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.598638,0.000000
2,No log,0.601449,0.202356
3,0.557400,0.599454,0.322391
4,0.557400,0.701667,0.322646
5,0.342600,0.756454,0.318876


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-cola/run-3/checkpoint-214
Configuration saved in distilbert-base-uncased-finetuned-cola/run-3/checkpoint-214/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola/run-3/checkpoint-214/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/run-3/checkpoint-214/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/run-3/checkpoint-214/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/tokenizer_config.json
Special tokens file saved in distilbert-

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.594152,0.000000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-cola/run-4/checkpoint-107
Configuration saved in distilbert-base-uncased-finetuned-cola/run-4/checkpoint-107/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola/run-4/checkpoint-107/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/run-4/checkpoint-107/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/run-4/checkpoint-107/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/tokenizer_config.json
Special tokens file saved in distilbert-

In [34]:
best_run

BestRun(run_id='3', objective=0.31887574398387536, hyperparameters={'learning_rate': 6.44059884120768e-06, 'num_train_epochs': 5, 'seed': 9, 'per_device_train_batch_size': 4})

In [35]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.598638,0.000000
2,No log,0.601449,0.202356
3,0.557400,0.599454,0.322391
4,0.557400,0.701667,0.322646
5,0.342600,0.756454,0.318876


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-cola/checkpoint-214
Configuration saved in distilbert-base-uncased-finetuned-cola/checkpoint-214/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola/checkpoint-214/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/checkpoint-214/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/checkpoint-214/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-cola/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola/sp

TrainOutput(global_step=1070, training_loss=0.43860412847215885, metrics={'train_runtime': 1658.714, 'train_samples_per_second': 2.577, 'train_steps_per_second': 0.645, 'total_flos': 17727604860372.0, 'train_loss': 0.43860412847215885, 'epoch': 5.0})